In [1]:
from py2neo import Graph
graph = Graph("http://localhost:7474", auth=("neo4j", "990910lhylhy"))

In [2]:
kword1 = 'Lay (entertainer)'
kword2 = 'Exo (band)'

In [3]:
c1 = 'MATCH p=(n1{entityName:\''+kword1+'\'})-[*1..3]->(n2{entityName:\''+kword2+'\'}) return p'
s1 = graph.run(c1).data()

In [4]:
nodes = []
edges = []
for i in range(len(s1)):
    nodes = nodes + list(s1[i]['p'].nodes)
    edges = edges + list(s1[i]['p'].relationships)
elements = {"nodes": nodes, "edges": edges}

In [5]:
if len(s1)>10:
    temp_list = []
    for i in range(len(s1)):
        count = 0
        for j in range(len(s1[i]['p'].nodes)):
            count += float(s1[i]['p'].nodes[j]['entityImportance'])
        temp_list.append(count)

In [6]:
filtered = []
for i in range(10):
#     for i in temp_list:
    filtered.append(temp_list.index(max(temp_list)))
    del temp_list[i]

In [7]:
nodes = []
edges = []
for i in range(len(s1)):
    if i in filtered:
        nodes = nodes + list(s1[i]['p'].nodes)
        edges = edges + list(s1[i]['p'].relationships)

In [128]:
from collections import Counter
import numpy as np
import heapq
# c1 = 'MATCH p=(n1{entityName:\'' + kword1 + '\'})-[*0..2]->(n2{entityName:\'' + kword2 + '\'}) return p'
c1 = 'MATCH p=allShortestPaths((n1{entityName:\''+kword1+'\'})-[*]->(n2{entityName:\''+kword2+'\'})) return p'
### time start
s1 = graph.run(c1).data()
print(len(s1))
importance = []
relationship_types = []
diversity = []
if len(s1) > 20:
    # importance
    for i in range(len(s1)):
        count = 0
        for j in range(len(s1[i]['p'].nodes)):
            count += float(s1[i]['p'].nodes[j]['entityImportance'])
        importance.append(count)
        for j in range(len(list(s1[i]['p'].relationships))):
            relationship_types.append(list(s1[i]['p'].relationships[j].types())[0])
    # diversity
    type_dict = dict(Counter(relationship_types))
    for i in range(len(type_dict)):
        type_dict[list(type_dict.keys())[i]] = 1-(list(type_dict.values())[i]/len(s1))
    for i in range(len(s1)):
        count_diversity = 0
        for j in range(len(list(s1[i]['p'].relationships))):
            count_diversity += type_dict[list(s1[i]['p'].relationships[j].types())[0]]
        diversity.append(count_diversity)
    # integral
    filtered = []
    sum_value = list(1000000*np.array(importance)+np.array(diversity))
    print('sum_value1:', sum_value)
    max_number = heapq.nlargest(2, sum_value)
    for t in max_number:
        index = sum_value.index(t)
        filtered.append(index)
        sum_value[index] = 0
    print('importance:', importance)
    print('diverisity:', diversity)
    print('sum_value2:', sum_value)
else:
    filtered = [i for i in range(len(s1))]
nodes = []
edges = []
for i in range(len(s1)):
    if i in filtered:
        print(s1[i]['p'])
        nodes = nodes + list(s1[i]['p'].nodes)
        edges = edges + list(s1[i]['p'].relationships)

2
sum_value1: [1.9578809694415518, 1.9578809694415518]
importance: [1.4578809694415519e-06, 1.4578809694415519e-06]
diverisity: [0.5, 0.5]
sum_value2: [0, 0]
(_1465422)-[:bandMember {inverse: 'directed'}]->(_1625759)
(_1465422)-[:associatedAct {inverse: 'direct'}]->(_1625759)


In [ ]:
def get_node_edges_2_new_new(kword):
    kword = kword.replace(', ', ',')
    kword1 = kword.split(',')[0]
    kword2 = kword.split(',')[1]
    print(kword1,',',kword2)
    c1 = 'MATCH p=allShortestPaths((n1{entityName:\'' + kword1 + '\'})-[*]->(n2{entityName:\'' + kword2 + '\'})) return p'
    ### time start
    start = time.time()
    s1 = graph.run(c1).data()
    print(len(s1))
    importance = []
    relationship_types = []
    diversity = []
    if len(s1) > 10:
        # importance
        for i in range(len(s1)):
            count = 0
            for j in range(len(s1[i]['p'].nodes)):
                count += float(s1[i]['p'].nodes[j]['entityImportance'])
            importance.append(count)
            for j in range(len(list(s1[i]['p'].relationships))):
                relationship_types.append(list(s1[i]['p'].relationships[j].types())[0])
        # diversity
        type_dict = dict(Counter(relationship_types))
        for i in range(len(type_dict)):
            type_dict[list(type_dict.keys())[i]] = 1 - (list(type_dict.values())[i] / len(s1))
        for i in range(len(s1)):
            count_diversity = 0
            for j in range(len(list(s1[i]['p'].relationships))):
                count_diversity += type_dict[list(s1[i]['p'].relationships[j].types())[0]]
            diversity.append(count_diversity)
        # integral
        filtered = []
        sum_value = list(1000000 * np.array(importance) + np.array(diversity))
        print('sum_value1:', sum_value)
        max_number = heapq.nlargest(10, sum_value)
        for t in max_number:
            index = sum_value.index(t)
            filtered.append(index)
            sum_value[index] = 0
        print('importance:', importance)
        print('diverisity:', diversity)
        print('sum_value2:', sum_value)
    else:
        filtered = [i for i in range(len(s1))]
    nodes = []
    edges = []
    for i in range(len(s1)):
        if i in filtered:
            print(s1[i]['p'])
            nodes = nodes + list(map(buildNodes2, s1[i]['p'].nodes))
            edges = edges + list(map(buildEdges2, s1[i]['p'].relationships))
    elements = {"nodes": nodes, "edges": edges}
    print(elements)
    ### time end
    mid = time.time()
    print(mid - start)
    return elements